In [34]:
library(reshape2)

ERROR: Error in library(reshape2): there is no package called ‘reshape2’


In [24]:
train_df <- read.csv("./data/train_processed.csv")
train_df <- train_df[,-c(1)]


In [15]:
dim(train_df)

[1] 1457   51

In [16]:
xtabs(~GarageQual+GarageCond+GarageFinish, data=train_df)
xtabs(~BsmtCond+BsmtFinType1, data=train_df)

, , GarageFinish = Fin

          GarageCond
GarageQual  Ex  Fa  Gd NoGarage  Po  TA
  Ex         1   0   0        0   0   0
  Fa         0   0   0        0   0   1
  Gd         0   0   1        0   0   1
  NoGarage   0   0   0        0   0   0
  Po         0   0   0        0   0   0
  TA         0   1   0        0   0 344

, , GarageFinish = NoGarage

          GarageCond
GarageQual  Ex  Fa  Gd NoGarage  Po  TA
  Ex         0   0   0        0   0   0
  Fa         0   0   0        0   0   0
  Gd         0   0   0        0   0   0
  NoGarage   0   0   0       81   0   0
  Po         0   0   0        0   0   0
  TA         0   0   0        0   0   0

, , GarageFinish = RFn

          GarageCond
GarageQual  Ex  Fa  Gd NoGarage  Po  TA
  Ex         0   0   0        0   0   0
  Fa         0   0   0        0   0   0
  Gd         0   0   2        0   0   5
  NoGarage   0   0   0        0   0   0
  Po         0   0   0        0   0   0
  TA         0   0   3        0   0 412

, , GarageFinish 

        BsmtFinType1
BsmtCond ALQ BLQ GLQ LwQ  No Rec Unf
      Fa   1   3   0   8   0   8  25
      Gd  12   3  26   5   0   3  16
      No   0   0   0   0  37   0   0
      Po   0   1   0   0   0   0   1
      TA 207 141 389  61   0 122 388

In [25]:
train_df <- train_df[, -c(which(colnames(train_df) == "GarageFinish"),
                            which(colnames(train_df) == "Exterior2nd"),
                            which(colnames(train_df) == "GarageCond"),
                            which(colnames(train_df) == "BsmtFinType1"))]
dim(train_df)

[1] 1457   47

In [26]:
rmlse <- function(yhat, y) {
  n <- length(yhat)
  return(sqrt((1/n)*sum((yhat-y)^2)))
}

In [30]:
set.seed(564)
controlParameter=trainControl(method = "cv", number = 10, savePredictions = TRUE)

ERROR: Error in trainControl(method = "cv", number = 10, savePredictions = TRUE): could not find function "trainControl"
